In [ ]:
!pip install google-cloud-bigquery google-cloud-storage

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Initialize BigQuery client
project_id = "bigdatabowl2025"
client = bigquery.Client(project=project_id, location='us-central1')

In [ ]:
# List of table names and corresponding DataFrame names
tables = {
    "players": "players",
    "plays" : "plays_df",
    "tracking_week_1": "tracking_df"  #  Will use combined_tracking later
}

# Dataset details
dataset_id = "bdb_tables"

# Query each table and assign to DataFrame variables
for table_name, df_name in tables.items():
    query = f"SELECT * FROM `{client.project}.{dataset_id}.{table_name}`"
    print(f"Querying table: {table_name}")
    globals()[df_name] = client.query(query).to_dataframe()

    print(f"Loaded {len(globals()[df_name])} rows into DataFrame: {df_name}")

Querying table: players
Loaded 1697 rows into DataFrame: players
Querying table: plays
Loaded 16124 rows into DataFrame: plays_df
Querying table: tracking_week_1
Loaded 7104700 rows into DataFrame: tracking_df


In [ ]:
print(tracking_df.head())
print(plays_df.head())
print(players.head())

       gameId  playId  nflId    displayName  frameId   frameType  \
0  2022091112    1082  37139   Randall Cobb      194  AFTER_SNAP   
1  2022091112     251  42485    Adrian Amos      187  AFTER_SNAP   
2  2022091112    1186  43338    Jarran Reed      195  AFTER_SNAP   
3  2022091112    2952  44911  Rasul Douglas       72  AFTER_SNAP   
4  2022091112    3696  48181  Yosuah Nijman       70  AFTER_SNAP   

                              time jerseyNumber club playDirection       x  \
0 2022-09-11 21:15:04.800000+00:00           18   GB         right  118.59   
1        2022-09-11 20:32:21+00:00           31   GB         right  110.47   
2 2022-09-11 21:21:39.700000+00:00           90   GB          left   90.26   
3 2022-09-11 22:55:31.200000+00:00           29   GB         right   83.38   
4 2022-09-11 23:25:45.800000+00:00           73   GB          left   94.06   

       y     s     a   dis       o     dir event  
0   8.00  0.63  2.69  0.08  327.35  132.36    NA  
1  12.89  3.69  5.20

In [ ]:
tracking_df['nflId'] = tracking_df['nflId'].astype(str)  # Convert to string
players['nflId'] = players['nflId'].astype(str)          # Convert to string

tracking_players_df = tracking_df.merge(players[['nflId', 'position', 'collegeName']], how='left', on='nflId')

print("Merge complete.")

Merge complete.


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Define team colors, including football
colors = {
    'ARI': "#97233F", 'ATL': "#A71930", 'BAL': '#241773', 'BUF': "#00338D",
    'CAR': "#0085CA", 'CHI': "#C83803", 'CIN': "#FB4F14", 'CLE': "#311D00",
    'DAL': '#003594', 'DEN': "#FB4F14", 'DET': "#0076B6", 'GB': "#203731",
    'HOU': "#03202F", 'IND': "#002C5F", 'JAX': "#9F792C", 'KC': "#E31837",
    'LA': "#003594", 'LAC': "#0080C6", 'LV': "#000000", 'MIA': "#008E97",
    'MIN': "#4F2683", 'NE': "#002244", 'NO': "#D3BC8D", 'NYG': "#0B2265",
    'NYJ': "#125740", 'PHI': "#004C54", 'PIT': "#FFB612", 'SEA': "#69BE28",
    'SF': "#AA0000", 'TB': '#D50A0A', 'TEN': "#4B92DB", 'WAS': "#5A1414",
    'football': 'red'  # Football color set to saddlebrown
}

def animate_play(tracking_players_df, plays_df, gameId, playId):
    # Filter for the specific play
    selected_play_df = plays_df[(plays_df['playId'] == playId) & (plays_df['gameId'] == gameId)].copy()

    # Filter for the specific play in tracking data
    selected_tracking_df = tracking_players_df[(tracking_players_df['playId'] == playId) & (tracking_players_df['gameId'] == gameId)].copy()

    # Get unique frames and sort them
    sorted_frame_list = list(selected_tracking_df['frameId'].unique())  # Convert to list
    sorted_frame_list.sort()  # Sort the list

    # Determine play direction
    play_direction = selected_tracking_df['playDirection'].iloc[0]
    print(selected_play_df['absoluteYardlineNumber'])

    # Adjust player coordinates relative to the line of scrimmage
    if play_direction == 'left':
        line_of_scrimmage = 120 - selected_play_df['absoluteYardlineNumber'].values[0]
        first_down_marker = line_of_scrimmage + selected_play_df['yardsToGo'].values[0]
        selected_tracking_df['x'] = 120 - selected_tracking_df['x']  # Flip horizontally
        selected_tracking_df['y'] = 53.3 - selected_tracking_df['y']  # Flip vertically
    else:
        line_of_scrimmage = selected_play_df['absoluteYardlineNumber'].values[0]
        first_down_marker = line_of_scrimmage + selected_play_df['yardsToGo'].values[0]
        # selected_tracking_df['x'] = line_of_scrimmage + (selected_tracking_df['x'] - line_of_scrimmage)

    # Extract play details
    down = selected_play_df['down'].values[0]
    quarter = selected_play_df['quarter'].values[0]
    gameClock = selected_play_df['gameClock'].values[0]
    playDescription = selected_play_df['playDescription'].values[0]

    # Handle long play descriptions
    if len(playDescription.split(" ")) > 15 and len(playDescription) > 115:
        playDescription = " ".join(playDescription.split(" ")[0:16]) + "<br>" + " ".join(playDescription.split(" ")[16:])

    # Set up animation controls
    updatemenus_dict = [
        {
            "buttons": [
                {"args": [None, {"frame": {"duration": 100, "redraw": False}, "fromcurrent": True, "transition": {"duration": 0}}],
                 "label": "Play", "method": "animate"},
                {"args": [[None], {"frame": {"duration": 0, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}],
                 "label": "Pause", "method": "animate"}
            ],
            "direction": "left",
            "pad": {"r": 10, "t": 87},
            "showactive": False,
            "type": "buttons",
            "x": 0.1,
            "xanchor": "right",
            "y": 0,
            "yanchor": "top"
        }
    ]

    sliders_dict = {
        "active": 0,
        "yanchor": "top",
        "xanchor": "left",
        "currentvalue": {
            "font": {"size": 20},
            "prefix": "Frame:",
            "visible": True,
            "xanchor": "right"
        },
        "transition": {"duration": 100, "easing": "cubic-in-out"},
        "pad": {"b": 10, "t": 50},
        "len": 0.9,
        "x": 0.1,
        "y": 0,
        "steps": []
    }

    # Create animation frames
    frames = []
    for frameId in sorted_frame_list:
        data = []

        # Add yard markers
        data.append(go.Scatter(
            x=np.arange(20, 110, 10),
            y=[5] * len(np.arange(20, 110, 10)),
            mode='text',
            text=list(map(str, list(np.arange(20, 61, 10) - 10) + list(np.arange(40, 9, -10)))),
            textfont_size=30,
            textfont_family="Courier New, monospace",
            textfont_color="#ffffff",
            showlegend=False,
            hoverinfo='none'
        ))
        data.append(go.Scatter(
            x=np.arange(20, 110, 10),
            y=[53.5 - 5] * len(np.arange(20, 110, 10)),
            mode='text',
            text=list(map(str, list(np.arange(20, 61, 10) - 10) + list(np.arange(40, 9, -10)))),
            textfont_size=30,
            textfont_family="Courier New, monospace",
            textfont_color="#ffffff",
            showlegend=False,
            hoverinfo='none'
        ))

        # Add line of scrimmage and first down line
        data.append(go.Scatter(x=[line_of_scrimmage, line_of_scrimmage], y=[0, 53.5], line_dash='dash', line_color='blue', showlegend=False, hoverinfo='none'))
        data.append(go.Scatter(x=[first_down_marker, first_down_marker], y=[0, 53.5], line_dash='dash', line_color='yellow', showlegend=False, hoverinfo='none'))

        legend_entries = set()
        # Plot players and football
        for _, row in selected_tracking_df[selected_tracking_df['frameId'] == frameId].iterrows():
            color = colors['football'] if row['displayName'] == 'football' else colors.get(row['club'], '#000000')
            name = 'football' if row['displayName'] == 'football' else row['club']
            hover_text = 'Football' if row['displayName'] == 'football' else f"Name: {row['displayName']}<br>Position: {row['position']}<br>College: {row.get('collegeName', 'N/A')}"

            show_legend = name not in legend_entries
            legend_entries.add(name)

            data.append(go.Scatter(
                x=[row['x']], y=[row['y']], mode='markers',
                marker=dict(size=8 if row['displayName'] == 'football' else 10, color=color),
                name=name,
                hovertext=hover_text,
                hoverinfo="text" if row['displayName'] != 'football' else 'none',
                showlegend=show_legend
            ))

        # Add frame to slider
        slider_step = {
            "args": [[frameId], {"frame": {"duration": 10, "redraw": False}, "mode": "immediate", "transition": {"duration": 0}}],
            "label": str(frameId),
            "method": "animate"
        }
        sliders_dict["steps"].append(slider_step)
        frames.append(go.Frame(data=data, name=str(frameId)))

    layout = go.Layout(
        autosize=True,
        width=None,
        height=None,
        xaxis=dict(range=[0, 120], autorange=False, tickmode='array', tickvals=np.arange(10, 111, 5).tolist(), showticklabels=False),
        yaxis=dict(range=[0, 53.3], autorange=False, showgrid=False, showticklabels=False),
        plot_bgcolor='#00B140',
        title=f"GameId: {gameId}, PlayId: {playId}<br>{gameClock} {quarter}Q<br>{playDescription}",
        updatemenus=updatemenus_dict,
        sliders=[sliders_dict]
    )

    fig = go.Figure(data=frames[0]['data'], layout=layout, frames=frames[1:])

    # Add First Down Markers
    for y_val in [0, 53]:
        fig.add_annotation(
            x=first_down_marker, y=y_val, text=str(down), showarrow=False,
            font=dict(family="Courier New, monospace", size=16, color="black"),
            align="center", bordercolor="black", borderwidth=2, borderpad=4, bgcolor="#ff7f0e"
        )

    return fig

# Example usage:
fig = animate_play(tracking_players_df, plays_df, gameId=2022091113, playId=2009)
fig.show()

6545    45
Name: absoluteYardlineNumber, dtype: Int64
